*links*
https://stackoverflow.com/questions/33802336/visualizing-output-of-convolutional-layer-in-tensorflow/33816991#33816991
https://kratzert.github.io/2017/06/15/example-of-tensorflows-new-input-pipeline.html


# Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.learn import learn_runner
import pandas as pd
from os import path
from inputs import convert
from tensorflow.contrib.learn import ModeKeys
from tensorflow.contrib.data import Iterator


# Convolutional Block

In [2]:
def block(inputs, filters, name, dropout, is_training):
    conv1 = tf.layers.conv2d(inputs,
                            filters,
                            4,
                            padding="same",
                            activation=tf.nn.relu,
                            name="conv{}_1".format(name))
    conv2 = tf.layers.conv2d(conv1,
                            filters,
                            4,
                            padding="same",
                            activation=tf.nn.relu,
                            name="conv{}_2".format(name))
    conv3 = tf.layers.conv2d(conv2,
                            filters,
                            4,
                            padding="same",
                            activation=tf.nn.relu,
                            name="conv{}_3".format(name))
    pool = tf.layers.max_pooling2d(conv3,
                            pool_size=2,
                            strides=2,
                            name="pool{}".format(name))
    if filters == 64:
        ix = 260
        iy=260
        channels=64
        V = tf.slice(conv3,(0,0,0,0),(1,-1,-1,-1))
        V = tf.reshape(V,(iy,ix,channels))
        ix += 4
        iy += 4
        V = tf.image.resize_image_with_crop_or_pad(V, iy, ix)
        cy = 8
        cx = 8
        V = tf.reshape(V,(iy,ix,cy,cx)) 
        V = tf.transpose(V,(2,0,3,1))
        V = tf.reshape(V,(1,cy*iy,cx*ix,1))
        tf.summary.image("image_conv{}_3".format(name), V)
        
    dropout = tf.layers.dropout(pool, dropout, training=is_training, name="dropout{}".format(name))
    return dropout

# Architecture

In [3]:
def architecture(inputs, filters_size, filter_names, dropouts, output_size, is_training):
    with tf.variable_scope('CovNet'):
        
        inputs = tf.layers.batch_normalization(inputs, name="batch_normalization")
        
        for i,filter_size in enumerate(filters_size):
            with tf.name_scope("conv-maxpool-{}s".format(filter_names[i])):
                      inputs = block(inputs, filter_size, filter_names[i], dropouts[i], is_training)

        with tf.name_scope("conv-dense"):
            inputs = tf.reshape(inputs, [-1, 8 * 8 * 512])
            dense1 = tf.layers.dense(
                inputs,
                4096,
                activation=tf.nn.relu,
                name='dense1',
            )
            dropout = tf.layers.dropout(dense1, 0.5, training=is_training, name="dense_dropout")
            dense2 = tf.layers.dense(
                dropout,
                4096,
                activation=tf.nn.relu,
                name='dense2',
            )
            net = tf.layers.dense(
                dense2,
                output_size,
                name='dense3',
            )
    
    return net

# Loss, Metrics and Model

In [4]:
def get_train_op_fn(loss, params):
    return tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        optimizer=tf.train.AdamOptimizer,
        learning_rate=params.learning_rate
)

In [5]:
def get_eval_metric_ops(labels, predictions):
    return {
        'Accuracy': tf.metrics.accuracy(
            labels=tf.argmax(labels, 0),
            predictions=predictions,
            name='accuracy')
    }

In [6]:
def model_fn(features, labels, mode, params):
    filter_size = [64,128,256,512,512]
    filter_names = ['64','128','256','512a','512b']
    dropouts = [0.1, 0.2, 0.3, 0.5, 0.5]
    output_size = labels.shape[1]
    is_training = mode == ModeKeys.TRAIN
    logits = architecture(features, filter_size, filter_names, dropouts, output_size, is_training)
    predictions = tf.argmax(logits, axis = -1)
    loss = None
    train_op = None
    eval_metric_ops = {}
    if mode != ModeKeys.INFER:
        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=tf.cast(labels, tf.int32),
            logits=logits)
        train_op = get_train_op_fn(loss, params)
        eval_metric_ops = get_eval_metric_ops(labels, predictions)
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops
    )

# Estimator Iterator and Inputs

In [7]:
def get_estimator(run_config, params):
    return tf.estimator.Estimator(
        model_fn=model_fn,  # First-class function
        params=params,  # HParams
        config=run_config  # RunConfig
    )

In [8]:
class IteratorInitializerHook(tf.train.SessionRunHook):

    def __init__(self):
        super(IteratorInitializerHook, self).__init__()
        self.iterator_initializer_func = None

    def after_create_session(self, session, coord):
        self.iterator_initializer_func(session)

In [9]:
def get_inputs(batch_size, images_paths, labels):
    
    iterator_initializer_hook = IteratorInitializerHook()

    def train_inputs():
        """Returns training set as Operations.
        Returns:
            (features, labels) Operations that iterate over the dataset
            on every evaluation
        """
        with tf.name_scope('Training_data'):
            dataset, images, labls = convert(images_paths, labels)
            iterator = Iterator.from_structure(dataset.output_types,
                                   dataset.output_shapes)
            next_element = iterator.get_next()

            # create two initialization ops to switch between the datasets
            training_init_op = iterator.make_initializer(dataset)
            iterator_initializer_hook.iterator_initializer_func = lambda sess: sess.run(training_init_op)
#             iterator = dataset.make_initializable_iterator()
#             next_example, next_label = iterator.get_next()
#             # Set runhook to initialize iterator
#             iterator_initializer_hook.iterator_initializer_func = \
#                 lambda sess: sess.run(
#                     iterator.initializer,
#                     feed_dict={images: images,
#                                labels: labls})
#             # Return batched (features, labels)
            return next_element

    # Return function and hook
    return train_inputs, iterator_initializer_hook


# Experiment

In [10]:
def experiment_fn(run_config, params):
    DATA_PATH = '/run/media/backman/yay/dogbreed/'
    run_config = run_config.replace(save_summary_steps = 1, save_checkpoints_steps=params.min_eval_frequency)
    estimator = get_estimator(run_config, params)
    
    train_images, train_labels, valid_images, valid_labels = prepare_data()
    train_input_fn, train_input_hook = get_inputs(10, train_images, train_labels)
    eval_input_fn, eval_input_hook = get_inputs(10, valid_images, valid_labels)
    
    experiment = tf.contrib.learn.Experiment(
        estimator=estimator,  # Estimator
        train_input_fn=train_input_fn,  # First-class function
        eval_input_fn=eval_input_fn,  # First-class function
        train_steps=params.train_steps,  # Minibatch steps
        min_eval_frequency=params.min_eval_frequency,  # Eval frequency
        train_monitors=[train_input_hook],  # Hooks for training
        eval_hooks=[eval_input_hook],  # Hooks for evaluation
        eval_steps=None  # Use evaluation feeder until its empty
    )
    
    return experiment


In [11]:
def run_experiment(argv=None):
    """Run the training experiment."""
    # Define model parameters
    params = tf.contrib.training.HParams(
        learning_rate=0.002,
        n_classes=10,
        train_steps=5000,
        min_eval_frequency=100
    )

    # Set the run_config and the directory to save the model and stats
    run_config = tf.contrib.learn.RunConfig()
    run_config = run_config.replace(model_dir='model')

    learn_runner.run(
        experiment_fn=experiment_fn,  # First-class function
        run_config=run_config,  # RunConfig
        schedule="train_and_evaluate",  # What to run
        hparams=params  # HParams
)

# Lab

In [12]:
def prepare_data():
    DATA_HOME = '/Users/fulop/Downloads/dogbreed/'
    CSV = DATA_HOME+'labels.csv'
    TRAIN_PATH = DATA_HOME + 'sample/train/'
    VALID_PATH = DATA_HOME + 'sample/valid/'
    dfile = pd.read_csv(DATA_HOME+'labels.csv')
    dfile['breed'] = pd.Categorical(dfile['breed'])
    dfile['breed'].value_counts().plot(kind='bar')
    dfile['breed'] = dfile.breed.cat.codes
    dfile['type'] = dfile.apply(lambda f: 'train' if path.isfile(TRAIN_PATH+f['id']+'.jpg') else 'valid' if path.isfile(VALID_PATH+f['id']+'.jpg') else 'none', axis=1)
    valid = dfile[dfile['type']=='valid']
    train = dfile[dfile['type']=='train']
    train_images = train['id'].map(lambda name : "{}{}.jpg".format(TRAIN_PATH,name)).values
    valid_images = valid['id'].map(lambda name : "{}{}.jpg".format(VALID_PATH,name)).values
    train_labels = train['breed'].values.astype(np.int32)
    valid_labels = valid['breed'].values.astype(np.int32)
    
    return train_images, train_labels, valid_images, valid_labels

In [13]:
run_experiment()

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1147ae9b0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 1, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'model'}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from model/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into model/model.ckpt.
INFO:tensorflow:loss = 1.42105e+08, step = 2
ERROR:tensorflow:Model diverged with loss = NaN.


NanLossDuringTrainingError: NaN loss during training.